In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import html
import json

In [48]:
def getIndeksLink(links, cat_link, page, date=datetime.strftime(datetime.today(), '%Y-%m-%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY-mm-dd
    """
    print("page ", page)
    url = "https://www.idntimes.com/ajax/index?category="+cat_link+"&type=all&page="+str(page)+"&date="+date
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
        
    indeks = soup.findAll('div', class_="box-latest box-list ")
    if indeks:
        for post in indeks:
            link = [post.find('a', href=True)['href'], cat_link]
            links.append(link)
        links = getIndeksLink(links, cat_link, page+1, date)
        
    return links

In [78]:
def getDetailBerita(links):
    """
    Mengambil seluruh element dari halaman berita
    """
    all_articles = []
    for link in links[:2]:
        articles = {}
        #link
        url = link[0]
        print(url)
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")
        
        #extract scrip json ld
        scripts_all = soup.findAll('script', attrs={'type':'application/ld+json'})
#         print(len(scripts_all))
        scripts = json.loads(scripts_all[-2].text)
        scripts2 = json.loads(scripts_all[-1].text)
        
        #category
        articles['category'] = scripts2['itemListElement'][0]['item']['name']
        articles['subcategory'] = scripts2['itemListElement'][1]['item']['name']
        
        articles['url'] = url
        
        article = soup.find('section', class_="content-post clearfix")
        
        #extract date
        pubdate = scripts['datePublished']
        pubdate = pubdate[0:19].strip(' \t\n\r')
        articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%Y-%m-%dT%H:%M:%S"), '%Y-%m-%d %H:%M:%S')
        articles['id'] = int(datetime.strptime(pubdate, "%Y-%m-%dT%H:%M:%S").timestamp()) + len(url)
        
        #extract author
        articles['author'] = scripts['author']['name']
        
        #extract title
        articles['title'] = scripts['headline']
        
        #source
        articles['source'] = 'idntimes'
        
        #extract comments count
#         articles['comments'] = int(soup.find('span', class_="commentWidget-total").find('b').text.strip(' \t\n\r'))
        articles['comments'] = 0
    
        #extract tags
        tags = article.find('div', class_="content-post-topic").findAll('a')
        articles['tags'] = ','.join([x.text for x in tags])
        
        #extract images
        articles['images'] = scripts['image']['url']
        
        #extract detail
        detail = article.find('article', attrs={'id':'article-content'})
            
        #hapus div
        if detail.findAll('div'):
            for div in detail.findAll('div'):
                div.decompose()
            
        #hapus link sisip
        if detail.findAll('strong'):
            for b in detail.findAll('strong'):
                if ("baca juga" in b.text.lower()):
                    b.decompose()
            
        #extract content
        detail = BeautifulSoup(detail.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        articles['content'] = content
        print('memasukkan berita id ', articles['id'])
        all_articles.append(articles)
    return all_articles

In [63]:
links = getIndeksLink([], 'news', 1, '2018-08-01')

page  1
https://www.idntimes.com/ajax/index?category=news&type=all&page=1&date=2018-08-01
page  2
https://www.idntimes.com/ajax/index?category=news&type=all&page=2&date=2018-08-01
page  3
https://www.idntimes.com/ajax/index?category=news&type=all&page=3&date=2018-08-01
page  4
https://www.idntimes.com/ajax/index?category=news&type=all&page=4&date=2018-08-01
page  5
https://www.idntimes.com/ajax/index?category=news&type=all&page=5&date=2018-08-01
page  6
https://www.idntimes.com/ajax/index?category=news&type=all&page=6&date=2018-08-01


In [76]:
detail = getDetailBerita(links)

https://www.idntimes.com/news/indonesia/margith-juita-damanik/video-wanita-diusir-saat-nonton-jokowi-jenguk-kahiyang
memasukkan berita id  1483203777
https://www.idntimes.com/news/indonesia/indianamalia/1134-bencana-pada-2018-kemensos-targetkan-100-kampung-siaga-bencana
memasukkan berita id  1483203781


In [77]:
detail

[{'category': 'News',
  'subcategory': 'Indonesia',
  'url': 'https://www.idntimes.com/news/indonesia/margith-juita-damanik/video-wanita-diusir-saat-nonton-jokowi-jenguk-kahiyang',
  'pubdate': '1970-01-01 07:00:00',
  'id': 1483203777,
  'author': 'Margith Juita Damanik',
  'title': 'Video Wanita Diusir saat Nonton Jokowi Jenguk Kahiyang',
  'source': 'idntimes',
  'comments': 0,
  'tags': 'kahiyang ayu,jokowi,cucu jokowi,#2019GantiPresiden',
  'images': 'https://cdn.idntimes.com/content-images/post/20180801/9e5ad7eebcb5204ae69e8ecf9edae487_600x400.jpeg',
  'content': 'Jakarta, IDN Times - Keluarga besar Presiden Joko \'Jokowi\' Widodo hari ini mendapatkan anggota baru, dengan lahirnya puteri pertama dari pasangan Kahiyang Ayu-Bobby Nasution. Bayi perempuan ini menjadi cucu perempuan pertama Jokowi dan Iriana Joko Widodo.Di saat momen bahagia tersebut, siapa sangka ada sedikit peristiwa berbeda di luar rumah sakit. Beruntung, kejadian tersebut tidak mengganggu Kahiyang, bayinya, maupu